In [2]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlunsplit, urlencode
MAX_P = 1000

out_dir = 'livedoor'
os.makedirs(out_dir, exist_ok=True)

In [3]:
MAINPAGE_URL = 'mainpage_url.txt'

# get all topic pages


In [4]:
base_url = 'https://news.livedoor.com/'
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
req = requests.get(base_url, headers=agent) 
soup = BeautifulSoup(req.text, 'html.parser')

In [ ]:
nav = soup.find('div', attrs={'id': 'nav'})
topics = nav.find_all('li')
for topic in topics:
    url_e = topic.find('a', href=True)
    if url_e is not None:
        url = url_e['href']
        if url_e['href'] != base_url:
            with open(os.path.join(out_dir, MAINPAGE_URL), 'a+') as f:
                f.write(url + '\n')
        

# get child pages of each topic

In [ ]:
def get_child_url(soup):
#     print(soup)
    links = soup.find_all('ul', attrs={'class': 'child'})
#     print(links)
    child_urls = []
    for link in links:
        cats = link.find_all('li')
        for cat in cats:
            child_url = cat.find('a', href=True)['href']
            child_urls.append(child_url)
    return child_urls

In [ ]:
with open(os.path.join(out_dir, MAINPAGE_URL), 'r') as f:
    base_urls = f.read().splitlines()
for base_url in base_urls:
    print(base_url)
    topic_name = base_url.split('/')[-2]
    req = requests.get(base_url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    child_urls = get_child_url(soup)
    if len(child_urls) != 0:
        with open(os.path.join(out_dir, '{}.txt'.format(topic_name)), 'w') as f:
            for url in child_urls:
                f.write(url + '\n')
#     print(topic_name, child_urls)
#     break

# get all pages of each topic 

In [ ]:
def encode_page(url, p):
    param = urlencode({'p': p})
    return urlunsplit(("", "", url, param, ""))

In [ ]:
def get_page_url(base_url, p):
    print(base_url + "123")
    url = encode_page(base_url, p)
    print(url + "1233")
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    links = soup.find_all('li', attrs={'class': 'hasImg'})
#     print(links)
#     input()
    atc_urls = []
    for link in links:
        atc_url = link.find('a', href=True)['href']
        atc_urls.append(atc_url)
    return atc_urls

# Parent page



In [ ]:
from tqdm import tqdm
with open(os.path.join(out_dir, MAINPAGE_URL), 'r') as f:
    base_urls = f.read().splitlines()

for base_url in base_urls:
    print(base_url)
    paths = []
    topic_name = base_url.split('/')[-2]
#     if topic_name not in ['ent']:
#         continue
    out_topic_dir = os.path.join(out_dir, topic_name)
    os.makedirs(out_topic_dir, exist_ok=True)
    for p in tqdm(range(MAX_P)):
        urls = get_page_url(base_url, p+1)
#         print(urls)
#         input()
        if len(urls) == 0 or urls[-1] in paths:
            print(p, len(urls))
            break 
        paths.extend(urls)
        with open(os.path.join(out_topic_dir, '{}.txt'.format(topic_name)), 'a+') as f:
            for url in urls:
                f.write(url + '\n')

# Child page


In [ ]:
has_child_paths = os.listdir(out_dir)
has_child_paths = [path for path in has_child_paths if path.endswith('txt') and path != MAINPAGE_URL]
print(has_child_paths)

In [ ]:
for path in has_child_paths:
    with open(os.path.join(out_dir, path), 'r') as f:
        base_urls = f.read().splitlines()
    parent_name = path.split('.')[0]
    print('-'*10)
    print(parent_name)
    for base_url in base_urls:
        paths = []
        topic_name = base_url.split('/')[-2]
        print(topic_name)
        out_topic_dir = os.path.join(out_dir, parent_name, topic_name)
        os.makedirs(out_topic_dir, exist_ok=True)
        for p in range(MAX_P):
            urls = get_page_url(base_url, p+1)
            if len(urls) == 0 or urls[-1] in paths:
                print(p, len(urls))
                break 
            paths.extend(urls)
            with open(os.path.join(out_topic_dir, '{}.txt'.format(topic_name)), 'w') as f:
                for url in urls:
                    f.write(url + '\n')

# Crawl data

In [ ]:
dirnames = os.listdir(out_dir)
dirnames = [name for name in dirnames if os.path.isdir(os.path.join(out_dir, name)) and 'ipynb' not in name]

data_dir = 'data/livedoor'
os.makedirs(data_dir, exist_ok=True)

In [5]:
def get_documents(url):
    document = {
        'topic': '',
        'time': '',
        'vender': '',
        'header': '',
        'body': ''
    }
    req = requests.get(url, headers=agent)
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # get topic
    try:
        topics = []
        topics_part = soup.find('nav', attrs={'class': 'breadcrumbs'})
        if topics_part is not None:
            topics_tag = topics_part.find_all('span')
            for tag in topics_tag:
                span_tag = tag.find('a')
                if span_tag is not None:
                    topics.append(span_tag.text.strip())
            document['topic'] = '_'.join(topics)
    except Exception as e:
        print(e)
    
    # get time and vender
    try:
        meta_tag = soup.find('div', attrs={'class': 'articleMeta'})
        time_tag = meta_tag.find('time', attrs={'class': 'articleDate'})
        document['time'] = time_tag.text.strip()

        vender_tag = meta_tag.find('p', attrs={'class': 'articleVender'})
        document['vender'] = vender_tag.text.strip()
    except:
        print(url)
    
    # get header
    try:
        header = soup.find('h1', attrs={'itemprop': 'headline'}) 
        if header is not None:
            document['header'] = header.text.strip()

        # get text
        links = soup.find_all('span', attrs={'itemprop': 'articleBody'})
        text = ''
        for link in links:
            cats = link.find_all('p')
            for cat in cats:
                text += cat.text.strip()
        document['body'] = text
    except:
        pass
    
    return document

get_documents('https://news.livedoor.com/article/detail/20139833/')

{'topic': 'ニューストップ_IT 経済_経済総合',
 'time': '2021年5月5日 10時0分',
 'vender': '東洋経済オンライン',
 'header': '｢かぐや姫｣のラストシーンに隠された深い意味',
 'body': '竹取物語の最後のシーンにドーンと登場する富士山。そこに込められた意味とは（写真：うみぼうず／PIXTA）働き方改革の一環として耳目を集めるリモートワーク。在宅勤務に切り替えることが珍しくなくなった昨今だが、私もその1人だ。1年経ってようやく新しい日課に慣れてきた今、毎日絶対に欠かせないのは、朝の散歩。熱々のカフェラテを入手すべく、あえて家から少し離れたお店まで歩いて、30分ほどで戻ってくるというのがお決まりコースだが、こうした穏やかな日々を過ごしている中、意外な発見があった。この連載の記事一覧はこちら持ち帰り用のカップを片手に周りを見渡すと、景色の一角がふと目にとまる。澄み渡った空に浮かび上がるアイコニックな輪郭、白い山頂……。そう、わが家の近くの高台から富士山がくっきりと見えるのだ。引っ越してきてからすでに3年が経過しているというのに、最近までその絶景スポットを完全に見逃していた。わざわざ坂を上らないとわからないし、曇っていると見えないし、見落とすのも無理もない。とはいえ、この辺りに暮らす人々はきっと誰でもそれに気づいているはずだ。日本人はみんな富士山が大好きだもの。畏れられると同時に、敬われてきた山。人間の日々の営みを静かに見守り続ける、変わらぬ存在でありながら、活火山である以上、つねに変化の可能性を秘めて、恐ろしくても尊い。富士山というのは日本の象徴の1つであり、日本人の感性に深く根付いているものだとも言える。富士山から噴き上がる煙は人々の悲しみところが、その見慣れた形はずっと同じかと思うと、実はそうでもない。まず、平安初期の貞観大噴火で姿ががらりと変わり、その次は江戸中期に再び大規模な噴火があって、さらに変化したそうだ。そう考えると、空を見上げていた古代人は、私たちが見ているものとまったく違う形をしたものを見ていたのかもしれない。『万葉集』に収められている歌には、恋焦がれた人々の悲しみは富士山から噴き上がる煙に例えられることが多く、そのイメージこそが当時の活発な活動を物語っている。さらに、葛飾北斎によ

In [ ]:
from tqdm import tqdm
import pandas as pd
# f_out = open(os.path.join(data_dir, 'data.csv'), 'w', buffering=1)
# f_out.write('topic,time,vender,header,body\n')
# data = pd.DataFrame(columns=['topic', 'time', 'vender', 'header', 'body'])
i = 0
for name in dirnames:
#     if name in ['ranking', 'straight_news', 'trend', '52', 'dom', 'eco', 'ent', 'gourmet', 'love', 'main', 'social_reaction', 'special', 'sports', 'world']:
#         continue
    print('-'*10)
    print(name)
    paths = os.listdir(os.path.join(out_dir, name))
    txt_paths = [path for path in paths if path.endswith('txt')]
    
    # for parent
    data = pd.DataFrame(columns=['topic', 'time', 'vender', 'header', 'body'])
    # process parent article
    for path in txt_paths:
        print('PARENT', path)
        with open(os.path.join(out_dir, name, path), 'r') as f:
            article_urls = f.read().splitlines()
        for a_url in tqdm(article_urls):
            a_url = a_url.replace('topics', 'article') ###
            document = get_documents(a_url)
            data = data.append(document, ignore_index=True)
    data.to_csv(os.path.join(data_dir, name + '.csv'))
    
    # for children
    child_paths = [path for path in paths if os.path.isdir(os.path.join(out_dir, name, path))]
    if len(child_paths) == 0:
        continue
    data = pd.DataFrame(columns=['topic', 'time', 'vender', 'header', 'body'])
    for c_path in child_paths:
        print('CHILD', c_path)
        c_a_paths = os.listdir(os.path.join(out_dir, name, c_path))
        for path in c_a_paths:
            print(path)
            with open(os.path.join(out_dir, name, c_path, path), 'r') as f:
                article_urls = f.read().splitlines()
            for a_url in tqdm(article_urls):
                a_url = a_url.replace('topics', 'article') ###
                document = get_documents(a_url)
                data = data.append(document, ignore_index=True)
    data.to_csv(os.path.join(data_dir, name + '_child.csv'))
data.to_csv(os.path.join(data_dir, 'data.csv'), index=False)
print(data.head())